In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

In [2]:
from scipy.io import loadmat
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist

In [3]:

mat = loadmat('..\\results\\var_k\dblp\\res_dblp_exp_4_002_128.mat')
features_matrix = mat['embs']

mat = loadmat("../data/dblp.mat")
A = mat["network"]
labels_matrix = mat["group"]
labels_count = labels_matrix.shape[1]

print(labels_matrix)

print(type(labels_matrix))

groups = labels_matrix.toarray()

print(type(groups[:5, :5]))
print(groups[:5, :5])

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (11, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (24, 0)	1.0
  (25, 0)	1.0
  (28, 0)	1.0
  (30, 0)	1.0
  (31, 0)	1.0
  (32, 0)	1.0
  (33, 0)	1.0
  :	:
  (12806, 1)	1.0
  (12808, 1)	1.0
  (12809, 1)	1.0
  (12810, 1)	1.0
  (12811, 1)	1.0
  (12812, 1)	1.0
  (12815, 1)	1.0
  (12817, 1)	1.0
  (12818, 1)	1.0
  (12820, 1)	1.0
  (12821, 1)	1.0
  (12823, 1)	1.0
  (12827, 1)	1.0
  (12829, 1)	1.0
  (12830, 1)	1.0
  (12831, 1)	1.0
  (12832, 1)	1.0
  (12833, 1)	1.0
  (12834, 1)	1.0
  (12835, 1)	1.0
  (12837, 1)	1.0
  (12838, 1)	1.0
  (12839, 1)	1.0
  (12840, 1)	1.0
  (12841, 1)	1.0
<class 'scipy.sparse._csc.csc_matrix'>
<class 'numpy.ndarray'>
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]


In [4]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, gram_test, top_k_list):
        assert gram_test.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(gram_test))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [5]:
# Example data creation
n = 13326  # Number of items
num_groups = 2  # Number of groups
#groups = np.random.randint(0, 2, size=(n, num_groups))  # Random binary groups matrix
print(groups[:5])
#groups = sparse.random(n, num_groups, density=0.2, format='csr', data_rvs=np.ones).toarray()  # Sparse binary groups matrix with 20% density

# Creating DataFrame for features
df_features = pd.DataFrame(features_matrix, columns=[f'feature_{i}' for i in range(128)])
#print(df_features.head())

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(df_features, groups, test_size=0.3, random_state=42)

print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))
print(len(y_test))
print(len(y_train))
print(y_test[:5])


[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
3998
9328
[[0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [8]:
clf = TopKRanker(SVC(kernel="precomputed",cache_size=4096,probability=True))
gram = squareform( 1 - pdist(X_train, 'hamming'));
gram_test = 1 - cdist(X_test, X_train, 'hamming');

clf.fit(gram, y_train)

top_k_list = [len(l) for l in y_test]
preds = clf.predict(gram_test, top_k_list)
mlb = MultiLabelBinarizer(classes=range(2))
f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average='micro')

accuracy = accuracy_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), normalize=False) / y_test.size
recalls = recall_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=None)  # Returns recall for each class
f1s = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=None)  # Returns F1 score for each class

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Average Recall: {np.mean(recalls):.2f}")
print(f"Average F1 Score: {np.mean(f1s):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, preds))

Overall Accuracy: 0.49
Average Recall: 1.00
Average F1 Score: 1.00

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      2246
           1       0.99      0.95      0.97      1807

   micro avg       0.98      0.97      0.97      4053
   macro avg       0.98      0.96      0.97      4053
weighted avg       0.98      0.97      0.97      4053
 samples avg       0.98      0.97      0.97      4053



In [15]:
mat = loadmat('..\\results\\var_k\dblp\\res_dblp_exp_2_002_128.mat')
features_matrix = mat['embs']

mat = loadmat("../data/dblp.mat")
A = mat["network"]
labels_matrix = mat["group"]
labels_count = labels_matrix.shape[1]

print(labels_matrix)

print(type(labels_matrix))

groups = labels_matrix.toarray()

print(type(groups[:5, :5]))
print(groups[:5, :5])

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (11, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (24, 0)	1.0
  (25, 0)	1.0
  (28, 0)	1.0
  (30, 0)	1.0
  (31, 0)	1.0
  (32, 0)	1.0
  (33, 0)	1.0
  :	:
  (12806, 1)	1.0
  (12808, 1)	1.0
  (12809, 1)	1.0
  (12810, 1)	1.0
  (12811, 1)	1.0
  (12812, 1)	1.0
  (12815, 1)	1.0
  (12817, 1)	1.0
  (12818, 1)	1.0
  (12820, 1)	1.0
  (12821, 1)	1.0
  (12823, 1)	1.0
  (12827, 1)	1.0
  (12829, 1)	1.0
  (12830, 1)	1.0
  (12831, 1)	1.0
  (12832, 1)	1.0
  (12833, 1)	1.0
  (12834, 1)	1.0
  (12835, 1)	1.0
  (12837, 1)	1.0
  (12838, 1)	1.0
  (12839, 1)	1.0
  (12840, 1)	1.0
  (12841, 1)	1.0
<class 'scipy.sparse._csc.csc_matrix'>
<class 'numpy.ndarray'>
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]


In [16]:
# Example data creation
n = 13326  # Number of items
num_groups = 2  # Number of groups
#groups = np.random.randint(0, 2, size=(n, num_groups))  # Random binary groups matrix
print(groups[:5])
#groups = sparse.random(n, num_groups, density=0.2, format='csr', data_rvs=np.ones).toarray()  # Sparse binary groups matrix with 20% density

# Creating DataFrame for features
df_features = pd.DataFrame(features_matrix, columns=[f'feature_{i}' for i in range(128)])
#print(df_features.head())

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(df_features, groups, test_size=0.3, random_state=42)

print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))
print(len(y_test))
print(len(y_train))
print(y_test[:5])


[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
3998
9328
[[0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [17]:
clf = TopKRanker(SVC(kernel="precomputed",cache_size=4096,probability=True))
gram = squareform( 1 - pdist(X_train, 'hamming'));
gram_test = 1 - cdist(X_test, X_train, 'hamming');

clf.fit(gram, y_train)

top_k_list = [len(l) for l in y_test]
preds = clf.predict(gram_test, top_k_list)
mlb = MultiLabelBinarizer(classes=range(2))
f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average='micro')

accuracy = accuracy_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), normalize=False) / y_test.size
recalls = recall_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=None)  # Returns recall for each class
f1s = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=None)  # Returns F1 score for each class

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Average Recall: {np.mean(recalls):.2f}")
print(f"Average F1 Score: {np.mean(f1s):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, preds))

Overall Accuracy: 0.49
Average Recall: 1.00
Average F1 Score: 1.00

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2246
           1       0.98      0.96      0.97      1807

   micro avg       0.98      0.97      0.97      4053
   macro avg       0.98      0.97      0.97      4053
weighted avg       0.98      0.97      0.97      4053
 samples avg       0.98      0.97      0.98      4053

